<a href="https://colab.research.google.com/github/mlaricobar/solution-datathon-interbank-2020/blob/desarrollo/%5B04%5D%20Model%20Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONSTRUCCIÓN DE LOS MODELOS


### Instalación de Librerías necesarias

In [1]:
!pip install catboost

     |████████████████████████████████| 65.8MB 47kB/s 



### Lectura de Datos

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/'My Drive'/Competencias/'[02] Interbank-2020'

/gdrive/My Drive/Competencias/[02] Interbank-2020


In [4]:
!ls -lh '[01] Intermediate'

total 2.0G
-rw------- 1 root root 521M Dec 20 17:16 ds_rcc.csv
-rw------- 1 root root 185M Dec 18 16:51 ds_rcc_instit.csv
-rw------- 1 root root 204M Dec 18 16:51 ds_rcc_instit_test.csv
-rw------- 1 root root 576M Dec 20 17:18 ds_rcc_test.csv
-rw------- 1 root root  22M Dec 31 22:28 ds_se.csv
-rw------- 1 root root  24M Dec 31 22:28 ds_se_test.csv
-rw------- 1 root root  18M Dec 31 19:34 ds_sunat.csv
-rw------- 1 root root  20M Dec 31 19:34 ds_sunat_test.csv
-rw------- 1 root root 192M Dec 31 13:48 rcc_product_level.csv
-rw------- 1 root root 212M Dec 31 17:27 rcc_product_level_test.csv


In [5]:
import numpy as np
import pandas as pd

In [6]:
def optimizar_df(df):
  
  for col_ in df.select_dtypes(include=[int]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="unsigned")

  for col_ in df.select_dtypes(include=[float]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="float")
  
  return df

In [10]:
df_base_train = optimizar_df(pd.read_csv("[00] Data/y_train.csv"))
df_se = optimizar_df(pd.read_csv('[01] Intermediate/ds_se.csv'))
df_rcc = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc.csv"))
df_rcc_2 = optimizar_df(pd.read_csv("[01] Intermediate/rcc_product_level.csv"))
df_censo_train = optimizar_df(pd.read_csv("[00] Data/censo_train.csv"))#.drop_duplicates()
#df_sunat_train = optimizar_df(pd.read_csv("[01] Intermediate/ds_sunat.csv"))

df_base_test = optimizar_df(pd.read_csv("[00] Data/sample_submission.csv"))
df_se_test = optimizar_df(pd.read_csv('[01] Intermediate/ds_se_test.csv'))
df_rcc_test = optimizar_df(pd.read_csv("[01] Intermediate/ds_rcc_test.csv"))
df_rcc_test_2 = optimizar_df(pd.read_csv("[01] Intermediate/rcc_product_level_test.csv"))
df_censo_test = optimizar_df(pd.read_csv("[00] Data/censo_test.csv"))#.drop_duplicates()
#df_sunat_test = optimizar_df(pd.read_csv("[01] Intermediate/ds_sunat_test.csv"))

In [9]:
df_rcc_2.loc[df_rcc_2["key_value"].isin([19746, 21193,  23473,  38585,  60175,  62839])]

,key_value,cod_producto_unico_nunique,saldo_mean_mean,saldo_sum_mean,saldo_count_mean,saldo_last_mean,saldo_first_mean,saldo_max_mean,saldo_min_mean,condicion_mean_mean,condicion_sum_mean,condicion_count_mean,condicion_last_mean,condicion_first_mean,condicion_max_mean,condicion_min_mean,diff_prev_codmes_mean_mean,diff_prev_codmes_sum_mean,diff_prev_saldo_mean_mean,diff_prev_saldo_sum_mean,diff_prev_saldo_std_mean,diff_prev_condicion_mean_mean,diff_prev_condicion_sum_mean,diff_prev_condicion_std_mean,abs_diff_prev_saldo_mean_mean,abs_diff_prev_saldo_sum_mean,abs_diff_prev_saldo_std_mean,abs_diff_prev_condicion_mean_mean,abs_diff_prev_condicion_sum_mean,abs_diff_prev_condicion_std_mean,incr_flag_saldo_mean_mean,incr_flag_saldo_sum_mean,decr_flag_saldo_mean_mean,decr_flag_saldo_sum_mean,same_flag_saldo_mean_mean,same_flag_saldo_sum_mean,incr_flag_condicion_mean_mean,incr_flag_condicion_sum_mean,decr_flag_condicion_mean_mean,decr_flag_condicion_sum_mean,same_flag_condicion_mean_mean,same_flag_condicion_sum_mean,recency_mean,long_recency_mean,diff_months_mean,ratio_saldo_last_over_first_mean
19177,21193,2,0.944430,0.944430,1.000000,0.944430,0.944430,0.944430,0.944430,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
40889,19746,4,0.971928,0.971928,1.000000,0.971928,0.971928,0.971928,0.971928,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
134599,23473,18,0.988407,5.988593,5.944445,0.988641,0.990144,0.993161,0.986335,0.000000,0.000000,5.944445,0.000000,0.0,0.000000,0.0,1.000000,6.357143,-0.000482,-0.001932,NaN,0.000000,0.000000,NaN,0.001410,0.010882,NaN,0.000000,0.000000,NaN,0.059414,0.555556,0.328483,2.500000,0.243827,1.888889,0.059414,0.555556,0.328483,2.500000,0.243827,1.888889,2.722222,7.666667,4.944445,1.000152
231907,38585,4,0.934392,0.934392,1.000000,0.934392,0.934392,0.934392,0.934392,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
269113,62839,19,0.935310,5.161009,5.526316,0.937115,0.935347,0.937490,0.932871,0.245614,0.736842,5.526316,0.736842,0.0,0.736842,0.0,1.364074,7.266667,0.000934,0.002240,NaN,0.466667,0.933333,NaN,0.002521,0.009091,NaN,0.466667,0.933333,NaN,0.178708,1.263158,0.281419,1.842105,0.172807,1.421053,0.178708,1.263158,0.281419,1.842105,0.172807,1.421053,2.157895,7.894737,5.736842,1.001851
358021,60175,5,0.956865,9.143265,9.600000,0.996397,0.949236,0.996435,0.935060,1131.099976,13573.200195,9.600000,1197.599976,1064.0,1197.599976,1064.0,1.000000,8.600000,0.006737,0.047161,0.030174,12.145454,133.600006,0.369767,0.021408,0.149858,0.020706,12.145454,133.600006,0.369767,0.175000,1.400000,0.325000,2.600000,0.391667,4.600000,0.175000,1.400000,0.325000,2.600000,0.391667,4.600000,1.000000,9.600000,8.600000,1.046257


#### Consolidar Dataset

In [11]:
df_train = (df_base_train.merge(df_se, how="left", on=["key_value"])
                 .merge(df_rcc, how="left", on=["key_value"])
                 .merge(df_rcc_2, how="left", on=["key_value"])
                 .merge(df_censo_train, how="left", on=["key_value"])
                .reset_index(drop=True)
           )#.merge(df_rcc_inst, how="left", on=["key_value"])#.merge(df_rcc, how="left", on=["key_value"])
            #.merge(df_sunat_train, how="left", on=["key_value"])

df_test = (df_base_test.merge(df_se_test, how="left", on=["key_value"])
                     .merge(df_rcc_test, how="left", on=["key_value"])
                     .merge(df_rcc_test_2, how="left", on=["key_value"])
                     .merge(df_censo_test, how="left", on=["key_value"])
                     .reset_index(drop=True)
           )#.merge(df_rcc_inst_test, how="left", on=["key_value"])#.merge(df_rcc_test, how="left", on=["key_value"])
           #.merge(df_sunat_test, how="left", on=["key_value"])

In [12]:
df_train.head()

,key_value,target,edad,sexo,est_cvl,sit_lab,cod_ocu,ctd_hijos,flg_sin_email,ctd_veh,cod_ubi,lgr_vot,prv,dto,rgn,tip_lvledu,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_255_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_255_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_255_COUNT,...,estud_9,estud_10,profe_1,profe_2,profe_3,profe_4,profe_5,profe_6,profe_7,profe_8,profe_9,profe_10,acteco1,acteco2,acteco3,acteco4,acteco5,acteco6,acteco7,acteco8,acteco9,acteco10,acteco11,acteco12,acteco13,acteco14,acteco15,acteco16,acteco17,acteco18,acteco19,acteco20,acteco21,ocup_1,ocup_2,ocup_3,ocup_4,ocup_5,ocup_6,flag_censo
0,0,0,-0.983651,1.0,4.0,2.0,22.0,0.0,1.0,0.0,-0.070167,17.0,47.0,687.0,4.0,0.0,1.062248,3.186743,3,0.927645,0.927645,1,0.961819,2.885458,3,0.961558,2.884674,3,NaN,0.000000,0,NaN,0.000000,0,0.927880,0.927880,1,NaN,0.0,0,...,0.029839,-0.233718,-0.340184,-0.139198,0.273272,-0.275950,0.631916,0.435428,-0.689387,-0.84259,0.035022,-0.011010,0.366657,-0.368741,-0.215789,-0.273762,-0.338135,-0.466236,0.758428,-0.681634,0.204462,0.017362,-0.173108,0.496280,-0.330325,-0.323340,-0.232125,0.049331,-0.378997,0.218663,-0.030854,0.691940,-0.11823,1.250016,0.009111,-0.037295,-0.623164,0.062284,0.691940,0.0
1,1,0,0.000953,1.0,4.0,1.0,22.0,0.0,1.0,1.0,-0.738566,10.0,186.0,849.0,6.0,0.0,NaN,0.000000,0,0.927283,2.781850,3,NaN,0.000000,0,NaN,0.000000,0,0.937762,2.813286,3,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,0.531124,0.0,4.0,1.0,22.0,0.0,1.0,0.0,-0.297248,15.0,84.0,1405.0,1.0,7.0,NaN,0.000000,0,0.927242,0.927242,1,NaN,0.000000,0,NaN,0.000000,0,0.995254,0.995254,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,-0.562298,-0.382281,-0.340184,-0.477116,-0.548783,-0.621288,-0.784926,-0.151014,-0.776772,-0.84259,-0.810852,-0.205903,-0.237047,-0.368741,-0.640609,-0.273762,-0.338135,-0.804707,-0.765458,-0.793597,-0.810684,-0.421994,-0.402767,-0.331307,-0.542542,-0.658023,-0.456580,-0.699305,-0.487390,-0.543045,-0.583661,-0.644919,-0.11823,-0.639720,-0.851924,-0.601297,-0.786996,-0.605414,-0.644919,1.0
3,3,0,-1.135129,0.0,4.0,1.0,22.0,0.0,1.0,0.0,-1.203435,4.0,9.0,977.0,6.0,7.0,NaN,0.000000,0,0.927209,0.927209,1,NaN,0.000000,0,NaN,0.000000,0,1.095067,1.095067,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,-1.438084,0.0,4.0,1.0,22.0,0.0,1.0,0.0,0.043424,18.0,111.0,391.0,3.0,0.0,0.943753,2.831259,3,0.927597,2.782790,3,0.941664,2.824993,3,0.932079,2.796238,3,NaN,0.000000,0,0.929493,2.788478,3,0.936681,2.810044,3,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_train.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358487 entries, 0 to 358486
Columns: 658 entries, key_value to flag_censo
dtypes: float32(502), float64(12), uint32(1), uint8(143)
memory usage: 769.6 MB


In [14]:
df_test.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396666 entries, 0 to 396665
Columns: 658 entries, key_value to flag_censo
dtypes: float32(502), float64(153), uint32(1), uint8(2)
memory usage: 1.2 GB


In [15]:
for col_ in df_train.columns:
  if df_train.loc[df_train[col_] == np.inf, :].shape[0] > 0:
    print(col_)

for col_ in df_test.columns:
  if df_test.loc[df_test[col_] == np.inf, :].shape[0] > 0:
    print(col_)

In [16]:
avoid_columns = []
main_columns = ["key_value", "target"]
predictores = [c for c in df_train.columns if c not in (main_columns + avoid_columns)]

In [17]:
print(predictores)

['edad', 'sexo', 'est_cvl', 'sit_lab', 'cod_ocu', 'ctd_hijos', 'flg_sin_email', 'ctd_veh', 'cod_ubi', 'lgr_vot', 'prv', 'dto', 'rgn', 'tip_lvledu', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM', 'U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT', 'U03M_SALDO_MEAN_FOR_PRODUCTO_255

### Preparar Marco de Evaluación

In [18]:
nrows, nrows_train, nrows_test = (df_train.shape[0] + df_test.shape[0]), df_train.shape[0], df_test.shape[0]

print("Cantidad de filas: {0:,}\n\t- Train:\t{1:,} filas\t({2:}%)\n\t- Test:\t\t{3:,} filas\t({4:}%)".format(nrows,
                                                                                                            nrows_train,
                                                                                                            round((100 * (nrows_train/nrows)), 2),
                                                                                                            nrows_test,
                                                                                                            round((100 * (nrows_test/nrows)), 2)
                                                                                                            ))

Cantidad de filas: 755,153
	- Train:	358,487 filas	(47.47%)
	- Test:		396,666 filas	(52.53%)


In [19]:
df_train.head()

,key_value,target,edad,sexo,est_cvl,sit_lab,cod_ocu,ctd_hijos,flg_sin_email,ctd_veh,cod_ubi,lgr_vot,prv,dto,rgn,tip_lvledu,U03M_SALDO_MEAN_FOR_PRODUCTO_0_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_0_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_1_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_2_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_2_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_2_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_3_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_3_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_3_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_4_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_4_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_4_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_6_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_6_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_6_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_8_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_8_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_8_COUNT,U03M_SALDO_MEAN_FOR_PRODUCTO_255_MEAN,U03M_SALDO_MEAN_FOR_PRODUCTO_255_SUM,U03M_SALDO_MEAN_FOR_PRODUCTO_255_COUNT,...,estud_9,estud_10,profe_1,profe_2,profe_3,profe_4,profe_5,profe_6,profe_7,profe_8,profe_9,profe_10,acteco1,acteco2,acteco3,acteco4,acteco5,acteco6,acteco7,acteco8,acteco9,acteco10,acteco11,acteco12,acteco13,acteco14,acteco15,acteco16,acteco17,acteco18,acteco19,acteco20,acteco21,ocup_1,ocup_2,ocup_3,ocup_4,ocup_5,ocup_6,flag_censo
0,0,0,-0.983651,1.0,4.0,2.0,22.0,0.0,1.0,0.0,-0.070167,17.0,47.0,687.0,4.0,0.0,1.062248,3.186743,3,0.927645,0.927645,1,0.961819,2.885458,3,0.961558,2.884674,3,NaN,0.000000,0,NaN,0.000000,0,0.927880,0.927880,1,NaN,0.0,0,...,0.029839,-0.233718,-0.340184,-0.139198,0.273272,-0.275950,0.631916,0.435428,-0.689387,-0.84259,0.035022,-0.011010,0.366657,-0.368741,-0.215789,-0.273762,-0.338135,-0.466236,0.758428,-0.681634,0.204462,0.017362,-0.173108,0.496280,-0.330325,-0.323340,-0.232125,0.049331,-0.378997,0.218663,-0.030854,0.691940,-0.11823,1.250016,0.009111,-0.037295,-0.623164,0.062284,0.691940,0.0
1,1,0,0.000953,1.0,4.0,1.0,22.0,0.0,1.0,1.0,-0.738566,10.0,186.0,849.0,6.0,0.0,NaN,0.000000,0,0.927283,2.781850,3,NaN,0.000000,0,NaN,0.000000,0,0.937762,2.813286,3,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0,0.531124,0.0,4.0,1.0,22.0,0.0,1.0,0.0,-0.297248,15.0,84.0,1405.0,1.0,7.0,NaN,0.000000,0,0.927242,0.927242,1,NaN,0.000000,0,NaN,0.000000,0,0.995254,0.995254,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,-0.562298,-0.382281,-0.340184,-0.477116,-0.548783,-0.621288,-0.784926,-0.151014,-0.776772,-0.84259,-0.810852,-0.205903,-0.237047,-0.368741,-0.640609,-0.273762,-0.338135,-0.804707,-0.765458,-0.793597,-0.810684,-0.421994,-0.402767,-0.331307,-0.542542,-0.658023,-0.456580,-0.699305,-0.487390,-0.543045,-0.583661,-0.644919,-0.11823,-0.639720,-0.851924,-0.601297,-0.786996,-0.605414,-0.644919,1.0
3,3,0,-1.135129,0.0,4.0,1.0,22.0,0.0,1.0,0.0,-1.203435,4.0,9.0,977.0,6.0,7.0,NaN,0.000000,0,0.927209,0.927209,1,NaN,0.000000,0,NaN,0.000000,0,1.095067,1.095067,1,NaN,0.000000,0,NaN,0.000000,0,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1,-1.438084,0.0,4.0,1.0,22.0,0.0,1.0,0.0,0.043424,18.0,111.0,391.0,3.0,0.0,0.943753,2.831259,3,0.927597,2.782790,3,0.941664,2.824993,3,0.932079,2.796238,3,NaN,0.000000,0,0.929493,2.788478,3,0.936681,2.810044,3,NaN,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Construcción de Modelos

In [20]:
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

from scipy.optimize import differential_evolution

In [21]:
seed_val = 41

ctb_params = {'loss_function':'Logloss',
              'eval_metric':'AUC',
              'task_type': 'GPU',
              'verbose': 200,
              'random_seed': 1
             }

lgb_params = {
                "objective": "binary",
                "metric": "auc",
                "max_depth": 8,
                "learning_rate": 0.05,
                "bagging_seed": 41,
                "min_data_in_leaf": 20,
                "bagging_fraction": 0.7,
                "feature_fraction": 0.7,
                "bagging_freq": 5,
                "verbosity": -1
              }

In [22]:
k = 5 # n folds

def model_with_cv(df_train, df_test, target_class, predictores, df_val=None):

  ##################################################################
  ############# ETAPA DE PREDICCIÓN
  ##################################################################

  # Se instancian las variables que se obtendrán como resultado.
  results_models = []
  list_models_cv = []

  df_probs_train = df_train[["key_value", target_class]].copy().reset_index(drop=True)
  df_probs_test = df_test[["key_value"]].copy().reset_index(drop=True)

  if df_val is not None:
    df_probs_val = df_val[["key_value", target_class]].copy().reset_index(drop=True)
  else:
    df_probs_val = None
  # Generar los Folds
  skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)

  for i, (traincv, valcv) in enumerate(skf.split(df_train[predictores], df_train[target_class])):

      print("Iteracion {}".format(i))
      
      df_train_cv = df_train.loc[traincv, :].copy().reset_index(drop=True)
      df_val_cv = df_train.loc[valcv, :].copy().reset_index(drop=True)

      # Fase de Entrenamiento
      print("\t CLASIFICACIÓN")
      print("\t\t 1. CATBOOST")
      ## CatBoost
      cbt_model = CatBoostClassifier(**ctb_params)
      cbt_model.fit(df_train_cv[predictores], df_train_cv[target_class], eval_set=(df_val_cv[predictores], df_val_cv[target_class]), use_best_model=True)

      print("\t\t 2. LIGHTGBM")
      ## LightGBM
      lgtrain = lgb.Dataset(df_train_cv[predictores], label=df_train_cv[target_class])
      lgval = lgb.Dataset(df_val_cv[predictores], label=df_val_cv[target_class])
      lgb_model = lgb.train(lgb_params, lgtrain, 900, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, verbose_eval=20, evals_result={})

      # Fase de Scoring
      ## Clasificación
      probas_train_cbtm = cbt_model.predict_proba(df_train_cv[predictores])[:, 1]
      probas_val_cbtm = cbt_model.predict_proba(df_val_cv[predictores])[:, 1]

      probas_train_lgbm = lgb_model.predict(df_train_cv[predictores])
      probas_val_lgbm = lgb_model.predict(df_val_cv[predictores])

      # Cálculo de Métricas
      ## Clasificación
      metric_train_cbtm = roc_auc_score(df_train_cv[target_class], probas_train_cbtm)
      metric_val_cbtm = roc_auc_score(df_val_cv[target_class], probas_val_cbtm)

      metric_train_lgbm = roc_auc_score(df_train_cv[target_class], probas_train_lgbm)
      metric_val_lgbm = roc_auc_score(df_val_cv[target_class], probas_val_lgbm)

      if df_val is not None:
        probas_val_out_cbtm = cbt_model.predict_proba(df_val[predictores])[:, 1]
        metric_val_out_cbtm = roc_auc_score(df_val[target_class], probas_val_out_cbtm)

        probas_val_out_lgbm = lgb_model.predict(df_val[predictores])
        metric_val_out_lgbm = roc_auc_score(df_val[target_class], probas_val_out_lgbm)

      else:
        probas_val_out_cbtm = np.zeros((1,1))
        metric_val_out_cbtm = 0

        probas_val_out_lgbm = np.zeros((1,1))
        metric_val_out_lgbm = 0

      # Cálculo de probabilidades en la base de Train y Validacion
      df_probs_train.loc[valcv, "prob_cbtm"] = probas_val_cbtm
      df_probs_train.loc[valcv, "prob_lgbm"] = probas_val_lgbm
      df_probs_train.loc[valcv, "FOLD"] = "FOLD_{0}".format(i+1)

      
      df_probs_test["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_test[predictores])[:, 1]
      df_probs_test["prob_lgbm_{}".format(i)] = lgb_model.predict(df_test[predictores])

      if df_val is not None:
        df_probs_val["prob_cbtm_{}".format(i)] = cbt_model.predict_proba(df_val[predictores])[:, 1]
        df_probs_val["prob_lgbm_{}".format(i)] = lgb_model.predict(df_val[predictores])

        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, "auc_val_out_cbtm": metric_val_out_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, "auc_val_out_lgbm": metric_val_out_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4), round(df_val[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4), round(np.mean(probas_val_out_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1},\tVal Out: {2}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4), round(np.std(probas_val_out_cbtm),4)))

      else:

        # Almacenar los resultados
        results_models.append({"auc_train_cbtm": metric_train_cbtm, "auc_val_cbtm": metric_val_cbtm, ## CATBOOST
                          "auc_train_lgbm": metric_train_lgbm, "auc_val_lgbm": metric_val_lgbm, ## LIGHTGBM
                          "seed_val": seed_val})
        
        print("AVG METRICS CLASSIFICATION")
        print("Mean Target:\tTrain: {0},\tVal In: {1}".format(round(df_train_cv[target_class].mean(),4), round(df_val_cv[target_class].mean(),4)))
        print("Mean Probs:\tTrain: {0},\tVal In: {1}".format(round(np.mean(probas_train_cbtm),4), round(np.mean(probas_val_cbtm),4)))
        print("Std Probs:\tTrain: {0},\tVal In: {1}".format(round(np.std(probas_train_cbtm),4), round(np.std(probas_val_cbtm),4)))
        
        
      list_models_cv.append({"cbtm_model": cbt_model, "lgbm_model": lgb_model})
      
      print("CATBOOST REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_cbtm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_cbtm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)

      print("LIGHTGBM REPORT")
      if df_val is not None:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_out_lgbm, "target": df_val[target_class]})
      else:
        df_prob_target_tmp = pd.DataFrame({"prob": probas_val_lgbm, "target": df_val_cv[target_class]})

      df_prob_target_tmp = df_prob_target_tmp.groupby(pd.qcut(df_prob_target_tmp["prob"], q=10)).agg({"target": ["sum", "count", "mean"]}).reset_index(drop=False)
      df_prob_target_tmp.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_prob_target_tmp.columns]
      df_prob_target_tmp["target_%"] = round(df_prob_target_tmp["target_sum"] * 100.0 / df_prob_target_tmp["target_sum"].sum(), 2)
      print(df_prob_target_tmp)
      print("#" * 500)
  
  ##################################################################
  ############# PARA ETAPA DE OPTIMIZACIÓN
  ##################################################################

  if df_val is not None:
    df_probs_val["prob_avg_cbtm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_cbtm" in c]].mean(axis=1)
    df_probs_val["prob_avg_lgbm"] = df_probs_val[[c for c in df_probs_val.columns if "prob_lgbm" in c]].mean(axis=1)

  df_probs_test["prob_avg_cbtm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_cbtm" in c]].mean(axis=1)
  df_probs_test["prob_avg_lgbm"] = df_probs_test[[c for c in df_probs_test.columns if "prob_lgbm" in c]].mean(axis=1)

  
  return results_models, list_models_cv, df_probs_train, df_probs_val, df_probs_test

In [23]:
results, list_models, df_probs_train, df_probs_val, df_probs_test = model_with_cv(df_train, df_test, "target", predictores, None)

Iteracion 0
	 CLASIFICACIÓN
		 1. CATBOOST
Learning rate set to 0.044018
0:	learn: 0.7760926	test: 0.7806471	best: 0.7806471 (0)	total: 20.2ms	remaining: 20.2s
200:	learn: 0.8587740	test: 0.8553468	best: 0.8553468 (200)	total: 3.85s	remaining: 15.3s
400:	learn: 0.8669116	test: 0.8605169	best: 0.8605169 (400)	total: 7.51s	remaining: 11.2s
600:	learn: 0.8715242	test: 0.8627594	best: 0.8627594 (600)	total: 11.2s	remaining: 7.42s
800:	learn: 0.8751198	test: 0.8640492	best: 0.8640492 (800)	total: 14.7s	remaining: 3.66s
999:	learn: 0.8782542	test: 0.8649926	best: 0.8649926 (999)	total: 18.3s	remaining: 0us
bestTest = 0.864992559
bestIteration = 999
		 2. LIGHTGBM
Training until validation scores don't improve for 100 rounds.
[20]	training's auc: 0.837032	valid_1's auc: 0.835171
[40]	training's auc: 0.848027	valid_1's auc: 0.844455
[60]	training's auc: 0.855843	valid_1's auc: 0.850931
[80]	training's auc: 0.861673	valid_1's auc: 0.85557
[100]	training's auc: 0.865765	valid_1's auc: 0.858049
[

In [24]:
# First Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.878254,0.864993,0.927719,0.866271,41
1,0.878423,0.864160,0.927449,0.866562,41
2,0.878053,0.864858,0.920229,0.866549,41
3,0.877606,0.867833,0.919273,0.869289,41
4,0.878562,0.864014,0.919363,0.864704,41


In [ ]:
# First Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.834539,0.822712,0.859006,0.823846,41
1,0.835161,0.820945,0.855816,0.821616,41
2,0.835441,0.817861,0.857260,0.819050,41
3,0.835025,0.821256,0.851961,0.822491,41
4,0.835012,0.819481,0.872939,0.821429,41


In [ ]:
# Second Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.843651,0.829637,0.890753,0.831454,41
1,0.844369,0.827960,0.869049,0.829013,41
2,0.844213,0.825024,0.880458,0.827586,41
3,0.843544,0.828033,0.883655,0.830802,41
4,0.843915,0.827735,0.890578,0.830337,41


In [ ]:
# Third Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.837454,0.822779,0.875155,0.824367,41
1,0.837709,0.822735,0.871554,0.823324,41
2,0.838256,0.819201,0.881905,0.820629,41
3,0.837436,0.822897,0.872295,0.824793,41
4,0.837011,0.822719,0.881190,0.824380,41


In [ ]:
# Fourth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.848285,0.833916,0.898653,0.836538,41
1,0.848607,0.832712,0.890521,0.834790,41
2,0.848699,0.830145,0.891555,0.832881,41
3,0.848064,0.831962,0.897827,0.835895,41
4,0.848851,0.831197,0.898466,0.833766,41


In [ ]:
# Fifth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.854373,0.842748,0.890797,0.844431,41
1,0.855238,0.840793,0.892966,0.842644,41
2,0.855101,0.836793,0.903635,0.840386,41
3,0.854775,0.839483,0.902767,0.842496,41
4,0.855069,0.839889,0.903772,0.841770,41


In [ ]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.949859,0.948399,0.968543,0.966217,41
1,0.949407,0.947514,0.968718,0.966161,41
2,0.949645,0.947987,0.968701,0.965941,41
3,0.949299,0.948137,0.968743,0.966143,41
4,0.949288,0.947315,0.968823,0.966365,41


In [22]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.860083,0.848422,0.892553,0.849547,41
1,0.860588,0.845172,0.896773,0.846512,41
2,0.860836,0.841588,0.903166,0.845084,41
3,0.860436,0.845567,0.910398,0.848112,41
4,0.860556,0.845273,0.900377,0.847654,41


In [29]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.860826,0.849093,0.894307,0.850258,41
1,0.861321,0.845266,0.894554,0.847733,41
2,0.861743,0.842110,0.912972,0.845521,41
3,0.861320,0.845963,0.912363,0.848544,41
4,0.861374,0.845834,0.912110,0.848061,41


In [45]:
# Sixth Version
pd.DataFrame(results)

,auc_train_cbtm,auc_val_cbtm,auc_train_lgbm,auc_val_lgbm,seed_val
0,0.878833,0.870579,0.918745,0.871590,41
1,0.879513,0.865097,0.918189,0.866995,41
2,0.879361,0.864071,0.920135,0.865900,41
3,0.879635,0.865029,0.928408,0.867146,41
4,0.879792,0.866854,0.928434,0.869142,41


### Importancia de Variables

In [25]:
df_predictores_ = pd.DataFrame({"name": predictores, "importance_fold_0": list_models[0]["cbtm_model"].get_feature_importance()})
for i in range(1, 5):
    df_predictores_ = df_predictores_.merge(pd.DataFrame({"name": predictores, "importance_fold_{0}".format(i): list_models[i]["cbtm_model"].get_feature_importance()}), how="left", on="name")

df_predictores_["importance_folds_mean"] = df_predictores_[[c for c in df_predictores_.columns if c not in ["name"]]].mean(axis=1)
df_predictores_ = df_predictores_.sort_values(by="importance_folds_mean", ascending=False).reset_index(drop=True)
df_predictores_.head(30)

,name,importance_fold_0,importance_fold_1,importance_fold_2,importance_fold_3,importance_fold_4,importance_folds_mean
0,long_recency_mean,6.022698,5.759596,6.075596,6.000044,6.340000,6.039587
1,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_SUM,3.925419,6.229312,3.435702,3.214646,6.130110,4.587038
2,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_COUNT,4.965869,2.993015,4.493487,4.852214,3.594352,4.179787
3,U03M_COUNT_OF_SALDO_MEAN_FOR_ALL_CONDICION_CAT...,3.954314,3.214051,2.869569,2.717309,2.828659,3.116781
4,cod_producto_unico_nunique,2.645303,2.740182,2.812468,2.749470,2.489252,2.687335
5,U03M_SALDO_MEAN_FOR_PRODUCTO_1_SUM,2.016168,2.117922,2.410077,2.586976,2.162168,2.258662
6,U03M_SALDO_MEAN_FOR_PRODUCTO_TOP_PROPENSITY_MEAN,1.463552,1.937199,1.732035,1.917203,1.669883,1.743975
7,matpis7,2.019664,2.185071,0.799959,1.345284,1.985698,1.667135
8,U03M_SALDO_MEAN_FOR_PRODUCTO_1_MEAN,1.509127,1.383303,1.351932,1.332326,1.543934,1.424124
9,U03M_SALDO_MEAN_FOR_PRODUCTO_0_COUNT,0.664676,1.620060,1.453852,1.894086,1.340514,1.394638


### Estabilidad de las Variables

In [47]:
def func_var_porc(n, d):
  try:
    return round(100.0 * (n/d - 1), 2)
  except:
    return None

In [51]:
df_predictores_["train_mean"] = df_predictores_["name"].apply(lambda c: df_train[c].mean())
df_predictores_["train_std"] = df_predictores_["name"].apply(lambda c: df_train[c].std())
df_predictores_["test_mean"] = df_predictores_["name"].apply(lambda c: df_test[c].mean())
df_predictores_["test_std"] = df_predictores_["name"].apply(lambda c: df_test[c].std())

df_predictores_["var_porc_mean_test_train"] = df_predictores_.apply(lambda row: func_var_porc(row["test_mean"], row["train_mean"]), axis=1)
df_predictores_["var_porc_std_test_train"] = df_predictores_.apply(lambda row: func_var_porc(row["test_std"], row["train_std"]), axis=1)

df_predictores_["abs_var_porc_mean_test_train"] = abs(df_predictores_["var_porc_mean_test_train"])

In [52]:
df_predictores_.shape, df_predictores_.loc[df_predictores_["abs_var_porc_mean_test_train"] < 10].shape

((677, 14), (362, 14))

In [53]:
new_predictores = df_predictores_.loc[df_predictores_["abs_var_porc_mean_test_train"] < 10, "name"].tolist()

### Interpretación

### Generando Submit

In [26]:
df_probs_test[["key_value", "prob_avg_cbtm"]].rename(columns={"prob_avg_cbtm": "target"}).to_csv('[02] Output/submit_02.csv', index=False)

In [27]:
df_probs_test.shape

(396666, 13)

In [1]:
!ls -lh '[02] Output'

ls: cannot access '[02] Output': No such file or directory


------